In [1]:
# Importing necessary modules

import sklearn.datasets as datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import tree
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import pandas as pd 
# read text file into pandas DataFrame and 
# create header 
import os
Column_names = ["TIME_str", "TIME_sec", "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
            "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN", "Vertical_label",\
           "Horizontal_label", "Speed_label", "Turn_rate_deg_per_sec"]
Target_category_name = "Horizontal_label"
data_name = "HOLD 2"
cwd = os.getcwd()
mypath = cwd + f"/../data/{data_name}/"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
count = 0

for file_name in onlyfiles:
    if "Traj" in file_name:
        if count == 0:
            file_path = mypath + file_name
            df = pd.read_csv(file_path, sep=",", header=None)
            df.columns = Column_names
            X_df = df[[ "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN",  "Turn_rate_deg_per_sec"]]
            Y_target = df[Target_category_name]
        else:
            file_path = mypath + file_name
            df_temp = pd.read_csv(file_path, sep=",", header=None)
            df_temp.columns = Column_names
            
            X_df_temp = df_temp[[ "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN",  "Turn_rate_deg_per_sec"]]
            Y_target_temp = df_temp[Target_category_name]
            X_df = pd.concat([X_df, X_df_temp])
            Y_target = pd.concat([Y_target, Y_target_temp])
        count +=1
        # if count >= 10:
        #     break
X_df["Turn_rate_deg_per_sec"] = X_df["Turn_rate_deg_per_sec"]*10

In [4]:
target_names =  list(set(list(Y_target)))
word_map_to_num = set(list(Y_target))
print("category:")
word_map_to_num


category:


{' Flying opposite established course',
 ' Hold Left Turn -1.5DegPerSec',
 ' Hold Left Turn -3.0DegPerSec',
 ' Hold Right Turn 1.5DegPerSec',
 ' Hold Right Turn 3.0DegPerSec',
 ' MaintainCurrentHeading',
 ' No Conclusion: Horizontal'}

In [5]:
n_sample = len(X_df)
print("number of points:", n_sample)
X_df = X_df.head(n_sample)
Y_target = Y_target.head(n_sample)
Y = []
for y in Y_target:
    for idx, s in enumerate(word_map_to_num):
        if s == y:
            Y.append(idx)
Y = np.array(Y)

number of points: 116214


In [6]:
import pandas as pd 
import dtreeviz
from sklearn.tree import export_text
# read text file into pandas DataFrame and 
# create header 
import os
cwd = os.getcwd()

# file_name_test = "Traj_AAL80-11121269.dat" # worst
file_name_test = "Traj_AAL245-11147798_POS_HOLD.dat" # normal

file_path_test = mypath + file_name_test


# file_name_test = "Traj_AAL1252-11172342POS_HOLD.dat"

# file_path_test = cwd + "/../data/HOLD/" + file_name_test


df_test = pd.read_csv(file_path_test, sep=",", header=None)
df_test.columns = Column_names
X_df_test = df_test[[  "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN",  "Turn_rate_deg_per_sec"]]

In [7]:
print("number of testing points:", len(X_df_test))

number of testing points: 3407


In [8]:
Z_target = df_test[Target_category_name]
n_sample = len(X_df_test)
X_df_test = X_df_test.head(n_sample)
Z_target = Z_target.head(n_sample)
Z = []
for z in Z_target:
    for idx, s in enumerate(word_map_to_num):
        if s == z:
            Z.append(idx)
Z = np.array(Z)
len(Z)

3407

## target 1: Horizontal_label

# Entropy

In [9]:
# Entropy, small tree with train data

criterion = "entropy"
max_depth = 6
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg

0.8045846455676596


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


In [10]:
# Entropy, small tree with train data
from sklearn.tree import export_text
from sklearn.tree import export_text_in_conditions

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg
r = export_text_in_conditions(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)



0.8045846455676596


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


If GROUNDSPEED_KTS <= 336.50 && GROUNDTRACK_DEG <= 159.50 && ALTITUDE_FT <= 4964.50 && Turn_rate_deg_per_sec <= 5.00 && Turn_rate_deg_per_sec <= -5.00 && GROUNDSPEED_KTS <= 191.50, then  class:  No Conclusion: Horizontal

If GROUNDSPEED_KTS <= 336.50 && GROUNDTRACK_DEG <= 159.50 && ALTITUDE_FT <= 4964.50 && Turn_rate_deg_per_sec <= 5.00 && Turn_rate_deg_per_sec <= -5.00 && GROUNDSPEED_KTS > 191.50, then  class:  No Conclusion: Horizontal

If GROUNDSPEED_KTS <= 336.50 && GROUNDTRACK_DEG <= 159.50 && ALTITUDE_FT <= 4964.50 && Turn_rate_deg_per_sec <= 5.00 && Turn_rate_deg_per_sec > -5.00 && GROUNDTRACK_DEG <= 43.50, then  class:  No Conclusion: Horizontal

If GROUNDSPEED_KTS <= 336.50 && GROUNDTRACK_DEG <= 159.50 && ALTITUDE_FT <= 4964.50 && Turn_rate_deg_per_sec <= 5.00 && Turn_rate_deg_per_sec > -5.00 && GROUNDTRACK_DEG > 43.50, then  class:  No Conclusion: Horizontal

If GROUNDSPEED_KTS <= 336.50 && GROUNDTRACK_DEG <= 159.50 && ALTITUDE_FT <= 4964.50 && Turn_rate_deg_per_sec > 5.00 &&

In [11]:
import graphviz
from sklearn.tree import export_graphviz
dot_data = export_graphviz(dtree_small_entropy,
                           feature_names=X_df.columns,  
                           class_names = target_names,

                           filled=True,  
                           rounded = True,
                            precision = 2,
                           proportion=True)
graph = graphviz.Source(dot_data)
graph.format = "png"
graph.render(f"{data_name}__all_{Target_category_name}_{criterion}" + "importances")

'HOLD 2__all_Horizontal_label_entropyimportances.png'

In [12]:
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)

In [13]:
print(r)

|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 159.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- GROUNDSPEED_KTS <= 191.50
|   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |--- GROUNDSPEED_KTS >  191.50
|   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |--- Turn_rate_deg_per_sec >  -5.00
|   |   |   |   |   |--- GROUNDTRACK_DEG <= 43.50
|   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |--- GROUNDTRACK_DEG >  43.50
|   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |--- Turn_rate_deg_per_sec >  5.00
|   |   |   |   |--- GROUNDTRACK_DEG <= 49.50
|   |   |   |   |   |--- ALTITUDE_FT <= 4810.50
|   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |--- ALTITUDE_FT >  4810.50
|   |   |   |   |   |   |--- class:  No C

In [14]:
tree = model.tree_

In [15]:
print(tree.children_left[tree.children_left[2]])

4


In [16]:
criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8879308861238749
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 159.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- GROUNDSPEED_KTS <= 191.50
|   |   |   |   |   |   |--- ALTITUDE_FT <= 272.50
|   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 7.50
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 5.50
|   |   |   |   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  5.50
|   |   |   |   |   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  7.50
|   |   |   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |   |--- ALTITUDE_FT >  272.50
|   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 17.50
|   |   |   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |   |   |   |--- GR

In [17]:
# Entropy, large tree with train data

criterion = "entropy"
dtree_large_n = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_n.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_n, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9939766293217684
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 159.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- GROUNDSPEED_KTS <= 191.50
|   |   |   |   |   |   |--- ALTITUDE_FT <= 272.50
|   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 130.50
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 8.50
|   |   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 6.50
|   |   |   |   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 43.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 4
|   |   |   |   |   |   |   |   |   |   |--- GROUNDTRACK_DEG >  43.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  6.50
|   |   |   |   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 50.00
|   |   |   |   |   |   |   |   |   |   |   |--- class:  No Conc

In [18]:
# Entropy, large tree with test data

criterion = "entropy"
dtree_large_entropy = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8858233049603756
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 159.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- GROUNDSPEED_KTS <= 191.50
|   |   |   |   |   |   |--- ALTITUDE_FT <= 272.50
|   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 130.50
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 8.50
|   |   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 6.50
|   |   |   |   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 43.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 4
|   |   |   |   |   |   |   |   |   |   |--- GROUNDTRACK_DEG >  43.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  6.50
|   |   |   |   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 50.00
|   |   |   |   |   |   |   |   |   |   |   |--- class:  No Conc

In [19]:
# Entropy, small tree with test data

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.6586439683005576
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 159.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- GROUNDSPEED_KTS <= 191.50
|   |   |   |   |   |   |--- ALTITUDE_FT <= 272.50
|   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 7.50
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 5.50
|   |   |   |   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  5.50
|   |   |   |   |   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  7.50
|   |   |   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |   |--- ALTITUDE_FT >  272.50
|   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 17.50
|   |   |   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |   |   |   |--- GR

In [20]:
# Entropy, large rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8949222189609627


In [21]:
# Entropy, small rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees, max_depth=5, min_samples_leaf=20)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.42735544467273257


# Gini

In [22]:
# Gini, small tree with train data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.7834168000413031
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 152.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |--- Turn_rate_deg_per_sec >  -5.00
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |--- Turn_rate_deg_per_sec >  5.00
|   |   |   |   |--- GROUNDTRACK_DEG <= 49.50
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |--- GROUNDTRACK_DEG >  49.50
|   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |--- ALTITUDE_FT >  4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |--- ALTITUDE_FT <= 10171.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |--- ALTITUDE_FT >  10171.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |--- Turn_rate_deg_per_sec >  -5.00
|   

In [23]:
# Gini, large tree with train data
criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9939766293217684
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 152.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- GROUNDTRACK_DEG <= 32.50
|   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 17.50
|   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 193.50
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 172.00
|   |   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 149.50
|   |   |   |   |   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  149.50
|   |   |   |   |   |   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  172.00
|   |   |   |   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  193.50
|   |   |   |   |   |   |   |   |--- class:  No Conclusion:

In [24]:
# Gini, large tree with test data

criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8591135896683298
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 152.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- GROUNDTRACK_DEG <= 32.50
|   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 17.50
|   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 193.50
|   |   |   |   |   |   |   |   |--- ALTITUDE_FT <= 2830.00
|   |   |   |   |   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -1312.00
|   |   |   |   |   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -1312.00
|   |   |   |   |   |   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |   |   |   |   |   |   |--- ALTITUDE_FT >  2830.00
|   |   |   |   |   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  193.50
|   |   |   |   |   |   |   |   |--- class:

In [25]:
# Gini, small tree with test data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.4294100381567361
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 152.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |--- Turn_rate_deg_per_sec >  -5.00
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |--- Turn_rate_deg_per_sec >  5.00
|   |   |   |   |--- GROUNDTRACK_DEG <= 49.50
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |--- GROUNDTRACK_DEG >  49.50
|   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |--- ALTITUDE_FT >  4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |--- ALTITUDE_FT <= 10171.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |--- ALTITUDE_FT >  10171.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |--- Turn_rate_deg_per_sec >  -5.00
|   

## Log_loss

In [26]:
# Log_loss, small tree with train data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_Log_loss, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.7716282031424785
|--- GROUNDSPEED_KTS <= 336.50
|   |--- GROUNDTRACK_DEG <= 159.50
|   |   |--- ALTITUDE_FT <= 4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= 5.00
|   |   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |--- Turn_rate_deg_per_sec >  -5.00
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |--- Turn_rate_deg_per_sec >  5.00
|   |   |   |   |--- GROUNDTRACK_DEG <= 49.50
|   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |--- GROUNDTRACK_DEG >  49.50
|   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |--- ALTITUDE_FT >  4964.50
|   |   |   |--- Turn_rate_deg_per_sec <= -5.00
|   |   |   |   |--- ALTITUDE_FT <= 11829.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |   |--- ALTITUDE_FT >  11829.00
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |   |--- Turn_rate_deg_per_sec >  -5.00
|   

In [27]:
# Log_loss, large tree with train data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9939766293217684


In [28]:
# Log_loss, large tree with test data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8825946580569415


In [29]:
# Log_loss, small tree with test data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.5506310537129439
